In [24]:
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar10
from keras.utils import np_utils
from keras import backend as K
import numpy
from keras import optimizers, Model
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler

In [25]:
seed = 7
numpy.random.seed(seed)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [26]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [27]:
base_model = ResNet50(weights =None,input_shape=X_train.shape[1:],include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 12:
        lr *= 1e-2
    elif epoch > 8:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=numpy.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler]

In [28]:
Adam = optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 41s 812us/step - loss: 4.0182 - acc: 0.1401 - val_loss: 3.3727 - val_acc: 0.2057
Epoch 2/100
50000/50000 [==============================] - 17s 332us/step - loss: 2.6436 - acc: 0.1984 - val_loss: 2.2576 - val_acc: 0.2645
Epoch 3/100
50000/50000 [==============================] - 17s 334us/step - loss: 2.2311 - acc: 0.2450 - val_loss: 2.0090 - val_acc: 0.2984
Epoch 4/100
50000/50000 [==============================] - 17s 338us/step - loss: 2.0302 - acc: 0.2858 - val_loss: 1.9347 - val_acc: 0.3274
Epoch 5/100
50000/50000 [==============================] - 17s 339us/step - loss: 1.8952 - acc: 0.3315 - val_loss: 1.8510 - val_acc: 0.3467
Epoch 6/100
50000/50000 [==============================] - 17s 340us/step - loss: 1.7882 - acc: 0.3650 - val_loss: 1.7396 - val_acc: 0.3716
Epoch 7/100
50000/50000 [==============================] - 17s 340us/step - loss: 1.6742 - acc: 0.4055 - val_l

Epoch 59/100
50000/50000 [==============================] - 16s 326us/step - loss: 0.0804 - acc: 0.9740 - val_loss: 2.8632 - val_acc: 0.4877
Epoch 60/100
50000/50000 [==============================] - 16s 317us/step - loss: 0.0821 - acc: 0.9731 - val_loss: 2.7376 - val_acc: 0.4920
Epoch 61/100
50000/50000 [==============================] - 16s 318us/step - loss: 0.0808 - acc: 0.9730 - val_loss: 2.7728 - val_acc: 0.5016
Epoch 62/100
50000/50000 [==============================] - 16s 318us/step - loss: 0.0930 - acc: 0.9698 - val_loss: 2.8756 - val_acc: 0.4799
Epoch 63/100
50000/50000 [==============================] - 16s 320us/step - loss: 0.3813 - acc: 0.8772 - val_loss: 2.1984 - val_acc: 0.4806
Epoch 64/100
50000/50000 [==============================] - 16s 328us/step - loss: 0.1157 - acc: 0.9627 - val_loss: 2.6272 - val_acc: 0.5079
Epoch 65/100
50000/50000 [==============================] - 17s 339us/step - loss: 0.0728 - acc: 0.9761 - val_loss: 2.7985 - val_acc: 0.5045
Epoch 66/100
